<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.23.5
Pandas:  1.5.2


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [33]:
#ANSWER:
bikes=pd.read_table("bikeshare.csv",header=0,delimiter=",")
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [15]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [16]:
df = pd.read_excel(r'C:\Users\Manish\Desktop\IOD\labs\modules_2\Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [19]:
#ANSWER
df = pd.read_excel(r'C:\Users\Manish\Desktop\IOD\labs\modules_2\Iris.xls', sheet_name = 'Data',skiprows=[1,2,3], usecols=[1,2,3,4])
df

,Petal_width,Petal_length,Sepal_width,Sepal_length
0,0.2,1.5,3.1,4.6
1,0.2,1.4,3.6,5.0
2,0.4,1.7,3.9,5.4
3,0.3,1.4,3.4,4.6
4,0.2,1.5,3.4,5.0
...,...,...,...,...
142,2.3,5.2,3.0,6.7
143,1.9,5.0,2.5,6.3
144,2.0,5.2,3.0,6.5
145,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [20]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [21]:
url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4     Abbotsford, BC       Canada  YXX
 ...              ...          ...  ...
 1574        Zanzibar     Tanzania  ZNZ
 1575        Zaragoza        Spain  ZAZ
 1576       Zhengzhou        China  CGO
 1577        Zhoushan        China  HSN
 1578          Zurich  Switzerland  ZRH
 
 [1579 rows x 3 columns]]

In [22]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,"Abbotsford, BC",Canada,YXX
...,...,...,...
1574,Zanzibar,Tanzania,ZNZ
1575,Zaragoza,Spain,ZAZ
1576,Zhengzhou,China,CGO
1577,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [23]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [38]:
#ANSWER
df=pd.DataFrame(bikes)
df.shape

(17379, 17)

What are the column names in `bikes`?

In [39]:
#ANSWER
list(df)

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

What are the data types of these columns?

In [40]:
#ANSWER
df.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [46]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [47]:
#ANSWER
bikes['season']

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [48]:
bikes[['season']] #This gives the different object

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [57]:
#ANSWER
print(df['atemp'].iloc[0:5])

0    0.2879
1    0.2727
2    0.2727
3    0.2879
4    0.2879
Name: atemp, dtype: float64


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [59]:
bikes[[0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [61]:
#ANSWER
print(bikes.iloc[0])

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed            0.0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object


What is the correct way to access the 2nd column of the DataFrame by its index?

In [64]:
#ANSWER
print(df.iloc[:, 1])

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object


## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [65]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [67]:
#ANSWER
df.isnull().values.any()

False

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [69]:
#ANSWER
df.isnull().values.any().sum()

0

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [85]:
#ANSWER
def scale(a,b):
 if b==0 : 
    print(np.nan)
 else: 
    print(a/b ) 
            

Apply the Pandas `isna` function to the following data objects:

In [86]:
scale(4,0)

nan


In [76]:
x = 2.3
y = np.nan
print(x, y)


2.3 nan


AttributeError: module 'pandas' has no attribute 'insa'

In [77]:
#ANSWER
print(pd.isna(x))
print(pd.isna(y))

False
True


In [78]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [79]:
#ANSWER
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

? ANSWER: This is used to create a string that considers pandas as NaN (Not a Number). by-default pandas consider #N/A, -NaN, -n/a, N/A, NULL etc as NaN value

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [87]:
#ANSWER
bikes['season'].nunique()

4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [92]:
#ANSWER
df[['instant','dteday','windspeed']].describe()

,instant,windspeed
count,17379.0000,17379.000000
mean,8690.0000,0.190098
std,5017.0295,0.122340
min,1.0000,0.000000
25%,4345.5000,0.104500
50%,8690.0000,0.194000
75%,13034.5000,0.253700
max,17379.0000,0.850700


Compute and print the overall minimum and maximum of the numeric data columns:

In [95]:
bikes_min, bikes_max = (bikes.min(), bikes.max())
bikes_min, bikes_max

(instant                1
 dteday        2011-01-01
 season                 1
 yr                     0
 mnth                   1
 hr                     0
 holiday                0
 weekday                0
 workingday             0
 weathersit             1
 temp                0.02
 atemp                0.0
 hum                  0.0
 windspeed            0.0
 casual                 0
 registered             0
 cnt                    1
 dtype: object,
 instant            17379
 dteday        2012-12-31
 season                 4
 yr                     1
 mnth                  12
 hr                    23
 holiday                1
 weekday                6
 workingday             1
 weathersit             4
 temp                 1.0
 atemp                1.0
 hum                  1.0
 windspeed         0.8507
 casual               367
 registered           886
 cnt                  977
 dtype: object)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [96]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [98]:
#ANSWER
bikes['atemp'].quantile([0.10,0.25,0.75,0.90])

0.10    0.2424
0.25    0.3333
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [99]:
type(bikes['atemp'])

pandas.core.series.Series

In [100]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
3503,3504,2011-05-30,2,0,5,21,1,1,0,1,0.74,0.6970,0.70,0.1343,42,107,149
13264,13265,2012-07-11,3,1,7,21,0,3,1,1,0.72,0.6667,0.58,0.2239,69,322,391
15711,15712,2012-10-21,4,1,10,20,0,0,0,1,0.48,0.4697,0.44,0.1940,35,160,195
809,810,2011-02-06,1,0,2,4,0,0,0,1,0.26,0.2273,0.60,0.3582,0,2,2
692,693,2011-02-01,1,0,2,5,0,2,1,3,0.14,0.2121,0.93,0.0000,0,3,3


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [102]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [ ]:
#ANSWER


Here is a random sample of `atemp_level`:

In [103]:
atemp_level.sample(5)          

2738     (0.25, 0.5]
11841    (0.5, 0.75]
6820     (0.5, 0.75]
3431     (0.5, 0.75]
4369     (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [104]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

3920     warm
9439     mild
17202    mild
12246    warm
370      cool
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [111]:
#ANSWER
bikes["atemp_level"]=atemp_level
print(bikes[bikes['atemp_level']=='mild'].head())

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  \
0           1  0.24  0.2879  0.81        0.0       3          13   16   
1           1  0.22  0.2727  0.80        0.0       8          32   40   
2           1  0.22  0.2727  0.80        0.0       5          27   32   
3           1  0.24  0.2879  0.75        0.0       3          10   13   
4           1  0.24  0.2879  0.75        0.0       0           1    1   

  atemp_level  
0        mild  
1        mild  
2        mild  
3        mild  
4        mil

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [106]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [112]:
# Cleaning Data
print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)
print(e.shape)

(3,)
(1, 3)
(2, 3, 4)
(4,)
(2, 4)


## Load Data

Load rock.csv and clean the dataset.

In [146]:
rock=pd.read_csv('rock.csv',na_values=['',' ','SONGFACTS.COM'], usecols = [0,1,2,4,6,7])
rock.head()

,Song Clean,ARTIST CLEAN,Release Year,First?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,1,82,82
1,Fantasy Girl,.38 Special,NaN,1,3,0
2,Hold On Loosely,.38 Special,1981.0,1,85,85
3,Rockin' Into the Night,.38 Special,1980.0,1,18,18
4,Art For Arts Sake,10cc,1975.0,1,1,1


## Check Column Names

Check column names and clean.

In [152]:
new_cols = []
for i, val in enumerate(rock.columns):
    col_name = str(val).lower().replace(' ','_').replace('?','').replace('*','')
    new_cols.append(col_name)

new_cols

['song_clean', 'artist_clean', 'release_year', 'first', 'playcount', 'fg']

In [158]:
list(new_cols)
rock.columns = new_cols

In [159]:
rock.columns

Index(['song_clean', 'artist_clean', 'release_year', 'first', 'playcount',
       'fg'],
      dtype='object')

## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [160]:
rock['release_year']= rock['release_year'].fillna(0)



## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [164]:
rock.dtypes

song_clean      object
artist_clean    object
release_year     int32
first            int64
playcount        int64
fg               int64
dtype: object

In [163]:
rock['release_year'] = rock['release_year'].astype('int')

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [165]:
rock.describe()

,release_year,first,playcount,fg
count,2230.000000,2230.0,2230.000000,2230.000000
mean,1465.331390,1.0,16.872646,15.048430
std,867.196161,0.0,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000
25%,0.000000,1.0,1.000000,0.000000
50%,1973.000000,1.0,4.000000,3.000000
75%,1981.000000,1.0,21.000000,18.000000
max,2014.000000,1.0,142.000000,142.000000


## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [169]:
rock.loc[rock['release_year'] < 1970]

,song_clean,artist_clean,release_year,first,playcount,fg
1,Fantasy Girl,.38 Special,0,1,3,0
10,"Baby, Please Don't Go",AC/DC,0,1,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,0,1,5,0
16,Girls Got Rhythm,AC/DC,0,1,24,0
24,Let's Get It Up,AC/DC,0,1,4,0
...,...,...,...,...,...,...
2216,"I'm Bad, I'm Nationwide",ZZ Top,0,1,10,0
2218,Just Got Paid,ZZ Top,0,1,2,0
2221,My Head's In Mississippi,ZZ Top,0,1,1,0
2222,Party On The Patio,ZZ Top,0,1,14,0


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [172]:
def convert_to_float(rock): 
    
    #converts cells in a dataframe to float and otherwise replaces them with np.nan.
    
    return rock.apply(pd.to_numeric, errors='coerce').fillna(np.nan)


## Apply these functions to your dataset

## Describe the new float-only DataFrame.

In [173]:
rock.describe()

,release_year,first,playcount,fg
count,2230.000000,2230.0,2230.000000,2230.000000
mean,1465.331390,1.0,16.872646,15.048430
std,867.196161,0.0,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000
25%,0.000000,1.0,1.000000,0.000000
50%,1973.000000,1.0,4.000000,3.000000
75%,1981.000000,1.0,21.000000,18.000000
max,2014.000000,1.0,142.000000,142.000000


>




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



